In [37]:
from quaternion_filters import quaternion
import numpy as np
from sympy import *
from collections import OrderedDict

In [2]:
def euler2q(roll, pitch, heading):
    q = quaternion.Quaternion.axis_angle(np.array([0, 0, 1.]), heading)
    q = q*quaternion.Quaternion.axis_angle(np.array([0, 1., 0]), pitch)
    q = q*quaternion.Quaternion.axis_angle(np.array([1., 0, 0]), roll)
    return q

def eulers(q):
    """
    returns: roll, pitch, heading
    """
    a, b, c, d = q.a, q.b, q.c, q.d
    phi = atan2(2*(a*b + c*d),
                        1-2*(b**2 + c**2))
    theta = asin(2*(a*c - d*b))
    psi = atan2(2*(a*d+b*c),
                        1-2*(c**2+d**2))
    return np.array([phi, theta, psi])


def normalize_heading_deg(val_deg):
    if val_deg > 360:
        return normalize_heading(val_deg - 360)
    elif val_deg < 0:
        return normalize_heading(val_deg + 360)
    else:
        return val_deg

In [7]:
roll, pitch, heading = symbols(['roll', 'pitch', 'heading'])
wx, wy, wz = symbols(['wx', 'wy', 'wz'])
ax, ay, az = symbols(['ax', 'ay', 'az'])
mx, my, mz = symbols(['mx', 'my', 'mz'])
G = symbols('G')
dt = symbols('dt')

state = [roll, pitch, heading, ax, ay, az, wx, wy, wz]

quaternion.arctan2 = atan2  # Make quaternion use symbolic functions
quaternion.arcsin = asin
quaternion.sqrt = sqrt
quaternion.sin = sin
quaternion.cos = cos


In [21]:
# Which way is ac for roll
def test():
    heading = 335*np.pi/180
    roll = 60*np.pi/180
    pitch = 0*5*np.pi/180
    q = euler2q(roll, pitch, heading)
    print(eulers(q)*180/np.pi)

    a = np.array([0,0,-G]) + np.array([-sin(heading), cos(heading), 0])*G*tan(roll)

    ab = (q.inv()*quaternion.Quaternion.from_vec(a)*q).as_ndarray()[1:]
    print("body a", ab)
#test()

# Do rotations go the right way?
def test():
    heading = 135*np.pi/180
    roll = 0*np.pi/180
    pitch = 0*5*np.pi/180
    q = euler2q(roll, pitch, heading)
    wx = 10*np.pi/180
    wy = 10*np.pi/180
    wz = 10*np.pi/180
    dt = .1
    qdelta = quaternion.Quaternion(1, wx/2*dt, wy/2*dt, wz/2*dt)
    #print(eulers(q)*180/np.pi)
    qnext = q * qdelta
    
    print("roll pitch heading", eulers(qnext))
    
test()

roll pitch heading [1.00892971901171 0.991322812393154
 -43.9822113375784 + 57.2957795130823*pi]


In [43]:
q = euler2q(roll, pitch, heading)

qdelta = quaternion.Quaternion(1, wx/2*dt, wy/2*dt, wz/2*dt)  # small angle approximation of rotation
qnext = q*qdelta

a = np.array([0,0,-G]) + np.array([-sin(heading), cos(heading), 0])*G*tan(roll)

ab = (q.inv()*quaternion.Quaternion.from_vec(a)*q).as_ndarray()[1:]

#wpredict = (q.inv()*quaternion.Quaternion.from_vec([0,0,-G/tas*tan(phi)])*q).as_ndarray()[1:]
wpredict = [wx, wy, wz]
statenext = np.concatenate([eulers(qnext), ab, wpredict])
#statenext = np.array([x.subs(q0**2+q1**2+q2**2+q3**2,1) for x in statenext])
for n in range(len(state)):
    print(f"{state[n]}_next = {statenext[n]}")

roll_next = atan2(2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(-dt*wx*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 - dt*wy*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - dt*wz*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + 1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2)) + 2*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/

In [44]:
F = np.zeros((len(state), len(state)), dtype=object)
for n in range(len(state)):
    for m in range(len(state)):
        tmp = (diff(statenext[n], state[m]))
        F[n, m] = tmp
        if tmp != 0:
            print(f"F[{n}, {m}] = {tmp}")

F[0, 0] = (-2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(-dt*wx*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 - dt*wy*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - dt*wz*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + 1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2)) - 2*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos

F[0, 2] = (-2*(dt*wx*(0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2)) + dt*wy*(-sin(heading/2)*cos(pitch/2)*cos(roll/2)/2 + 0.5*sin(pitch/2)*sin(roll/2)*cos(heading/2)) - dt*wz*(-0.5*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 0.5*sin(pitch/2)*cos(heading/2)*cos(roll/2)) - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + sin(roll/2)*cos(heading/2)*cos(pitch/2))*(dt*wx*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wy*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wz*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 + 1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2)) - 2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 

F[0, 8] = (-2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(-dt*wx*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 - dt*wy*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - dt*wz*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + 1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2)) - 2*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos

F[1, 6] = -(dt*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 + 1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2)) + dt*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(dt*wx*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wy*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wz*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 + 1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2)) - dt*(1.

F[2, 0] = (-2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(dt*wx*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wy*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wz*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 + 1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2)) - 2*(-dt*wx*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 - dt*wy*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos

...

F[4, 0] = (0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-G*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) + G*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*cos(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(head

F[4, 1] = (-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(G*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*sin(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - G*(-1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))*cos(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos

F[4, 2] = (-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) - 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-G*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))*sin(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - G*(-1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(he

F[5, 0] = (-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) - 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(G*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))*cos(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) + G*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(head

F[5, 2] = (0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(G*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*sin(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - G*(-1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))*cos(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(

In [47]:
subexpres = {sin(pitch/2): 0, cos(pitch/2): 1}
for m in range(len(state)):
    for n in range(len(state)):
        for k, v in subexpres.items():
            F[m, n] = F[m, n].subs(k, v)
            
subexprs, subs = cse(F.flatten())
subs = np.reshape(subs, (9, 9))

for se in subexprs:
    print(f"{se[0]} = {(se[1])}")
    
for n in range(len(state)):
    for m in range(len(state)):
        tmp = subs[n, m]
        if tmp != 0:
            print(f"F[{n}, {m}] = {(tmp)}")

x0 = heading/2
x1 = sin(x0)
x2 = roll/2
x3 = cos(x2)
x4 = x1*x3
x5 = sin(x2)
x6 = x1*x5
x7 = 0.5*x6
x8 = dt*x7
x9 = wx*x8
x10 = -x9
x11 = cos(x0)
x12 = x11*x5
x13 = x12/2
x14 = dt*wy
x15 = x11*x3
x16 = 0.5*x15
x17 = dt*wz
x18 = x16*x17
x19 = -x18
x20 = x10 - x13*x14 + x19 + x4
x21 = 1.0*x6
x22 = 0.5*x4
x23 = dt*x22
x24 = wx*x23
x25 = dt*x15
x26 = wy*x25
x27 = 0.5*x12
x28 = dt*x27
x29 = wz*x28
x30 = -x29
x31 = x21 + x24 + x26/2 + x30
x32 = 2*x31
x33 = -x20*x32
x34 = dt*wx
x35 = wy*x8
x36 = wz*x23
x37 = x35 + x36
x38 = -x13*x34 + x15 + x37
x39 = 1.0*x12
x40 = wx*x25
x41 = wz*x8
x42 = wy*x23
x43 = x41 - x42
x44 = x39 + x40/2 + x43
x45 = 2*x44
x46 = 1 - 2*x31**2
x47 = -2*x44**2 + x46
x48 = -wx*x28 + x15
x49 = -x35 + x48
x50 = -x36 + x49
x51 = x45*x50
x52 = 1.0*x4
x53 = wz*x25
x54 = wy*x28
x55 = x10 + x54
x56 = x52 + x53/2 + x55
x57 = x32*x56
x58 = 1/(x47**2 + (x51 + x57)**2)
x59 = x58*(-x51 - x57)
x60 = 2.0*x6
x61 = -x17*x39 + x26 + x34*x52 + x60
x62 = -x7
x63 = 0.25*x4
x64 = 0.25*x26
x65 

## Mag Update


Kalman equation:
$$y = z - h(x)$$
innovation = measurement - (func translating state to measurement)

In [38]:
q = euler2q(roll, pitch, heading)

mag_inertial = (q * quaternion.Quaternion.from_vec(np.array([mx, my, mz])) * q.inv()).as_ndarray()[1:]
mag_inertial[2] = 0
mag_inertial /= sqrt(mag_inertial[0]**2 + mag_inertial[1]**2)


mag_body = (q.inv() * quaternion.Quaternion.from_vec(mag_inertial) * q).as_ndarray()[1:]

# Only portion that we need to determine Jacobian is transfering "north vector"
# to body coordinates

h = (q.inv() * quaternion.Quaternion.from_vec(np.array([1.0, 0, 0])) * q).as_ndarray()[1:]

y = mag_body - h
H = np.zeros((2, len(state)), dtype=object)
for n in range(2):
    for m in range(len(state)):
        tmp = diff(h[n], state[m])
        H[n, m] = tmp
        if tmp != 0:
            tmp = tmp
            print(f"H[{n}, {m}] = {tmp}")           

H[0, 0] = -1.0*(-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) - 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - 1.0*(0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*

H[0, 2] = 1.0*(-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) - 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) + 1.0*(0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) + 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*c

H[1, 2] = 1.0*(-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) - 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - 1.0*(0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/

In [41]:

subexprs = {sin(pitch/2): 0, cos(pitch/2): 1.0}
for m in range(2):
    for n in range(len(state)):
        for k, v in subexprs.items():
            H[m, n] = H[m, n].subs(k, v)
subexprs, subs = cse(H.flatten())
subs = np.reshape(subs, (2, 9))   

for se in subexprs:
    print(f"{se[0]} = {(se[1])}")
    
for n in range(2):
    for m in range(len(state)):
        tmp = subs[n, m]
        if tmp != 0:
            print(f"H[{n}, {m}] = {(tmp)}")  # ccode

x0 = roll/2
x1 = sin(x0)
x2 = x1**2
x3 = heading/2
x4 = sin(x3)
x5 = x4**2
x6 = 1.0*x5
x7 = x2*x6
x8 = cos(x0)
x9 = x8**2
x10 = x6*x9
x11 = cos(x3)
x12 = x11**2
x13 = 1.0*x12
x14 = x13*x2
x15 = x13*x9
x16 = 1/(x10 + x14 + x15 + x7)
x17 = 2.0*x16
x18 = x11*x4
x19 = x17*x18
x20 = x1*x8
x21 = x17*x20
H[0, 2] = -x19*x2 - x19*x9
H[1, 0] = 4.0*x16*x18*x20
H[1, 1] = x12*x21 - x21*x5
H[1, 2] = x10*x16 + x14*x16 - x15*x16 - x16*x7
